_Task 1: Inner Workings of ResNet-152_

_4. Transfer Learning_

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet152
import os
from PIL import Image


Using pre-trained ResNet-152.
Replacing the final classification layer to match CIFAR-10.
Freezing the rest of the backbone so that only the classification head is trained.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet152(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, 17)
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True
model = model.to(device)
print(device)

cuda


Preparing dataloaders

In [3]:


data_dir = './data/pets'
train_txt = os.path.join(data_dir, 'annotations/trainval.txt')
test_txt  = os.path.join(data_dir, 'annotations/test.txt')
img_dir   = os.path.join(data_dir, 'images/images')

def parse_txt(txt_file):
    items = []
    with open(txt_file) as f:
        for line in f:
            parts = line.strip().split()
            img_name = parts[0] + ".jpg"
            label = int(parts[1]) - 1  # zero-based
            img_path = os.path.join(img_dir, img_name)
            items.append((img_path, label))
    return items

train_items = parse_txt(train_txt)
test_items  = parse_txt(test_txt)


In [4]:
class OxfordPetDataset(Dataset):
    def __init__(self, items, transform=None):
        self.items = items
        self.transform = transform
    
    def __len__(self):
        return len(self.items)
    
    def __getitem__(self, idx):
        path, label = self.items[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label


In [5]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

trainset = OxfordPetDataset(train_items, transform=transform)
testset  = OxfordPetDataset(test_items, transform=transform)

trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
testloader  = DataLoader(testset, batch_size=256, shuffle=False, num_workers=8)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.02, momentum=0.9)

In [7]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = running_loss / total
    accuracy = 100. * correct / total
    return avg_loss, accuracy


def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    avg_loss = running_loss / total
    accuracy = 100. * correct / total
    return avg_loss, accuracy

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model, trainloader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, testloader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")